**Segundo Projeto Final**

**Disciplina Tópicos Contemporâneos**

**Discentes:**

**Valéria Cristina A. R. de Figueredo** (vcarf@cesar.school)

**Manuela de Lacerda Bezerra Carvalho** (mlbc@cesar.school)

**Objetivo do Projeto:** Construção de um RAG utilizando LangChain

Deverá ser desenvolvido um sistema RAG (Retrieval-Augmented Generation), utilizando a biblioteca LangChain para um ou mais documentos. Pode ser arquivos PDF, de texto, páginas da web, etc. O projeto será avaliado nos seguintes aspectos: escolha do documento, splitting do documento, criação de vector store retrieval, geração de respostas.

**DESCRIÇÃO DO DOCUMENTO**

O Projeto de Lei nº 2338/2023 (“PL de Inteligência Artificial) visa estabelecer diretrizes e normas para o uso de tecnologias de inteligência artificial no Brasil.

O objetivo é que se torne o marco Legal de Inteligência Artificial, assim como estabeleça direitos para proteger os cidadãos e implemente ferramentas de governança, que serão operadas por instituições responsáveis pela fiscalização e supervisão das tecnologias de IA.

A sua compreensão pelos agentes de inteligência artificial é de grande importância para buscar salvaguardar a população em geral dos efeitos danosos da I.A., bem como permitir o avanço da I.A. de forma segura.

O Substitutivo ao Projeto de Lei 2338/2023 (“PL de IA”) foi aprovado pelo plenário do Senado Federal no dia 10/12/2024. O PL segue para a Câmara dos Deputados onde deverá ser analisado para aprovação e sanção ou, caso sofra alterações, voltar ao Senado para deliberação final.

A Comissão Temporária sobre Inteligência Artificial do Senado Federal aprovou o Substitutivo 199 ao PL. Uma das alterações foi a exclusão da classificação como sistemas de IA de alto risco aqueles utilizados para "curadoria, difusão, recomendação e distribuição, em grande escala e de forma significativamente automatizada, de conteúdo" — direcionados às plataformas digitais.

Assim como ocorreu essa mudança, outras ainda podem ocorrer. Daí a importância dos agentes de Inteligência Artificial e da população em geral terem conhecimento do andamento desse projeto de lei para fins de participação de cunho democrático, cobranças e melhorias.

# **1. Importar Bibliotecas**

In [1]:
! pip install langchain langchain_openai langchain_community
! pip install faiss-cpu
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchai

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os
import getpass



In [3]:
# Solicita a chave da API do OpenAI
openai_api_key = getpass.getpass(prompt="Digite sua chave da API OpenAI: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Digite sua chave da API OpenAI: ··········


## **Retrieval Augmented Generation (RAG)**

# **2. Carregando Documentos - Loading**

Carregamento do documento usando PyPDFLoader.

In [4]:
# Carregar o conteúdo da página
file_path = (
    "https://legis.senado.leg.br/sdleg-getter/documento?dm=9347622&ts=1735605216274&disposition=inline"
)

loader = PyPDFLoader(file_path)

# Carregar o conteúdo da página
docs = loader.load_and_split()

# Exibir o texto extraído
print(docs[1])

page_content='PROJETO DE LEI Nº       , DE 2023 
Dispõe sobre o uso da Inteligência Artificial. 
O CONGRESSO NACIONAL decreta: 
CAPÍTULO I 
DISPOSIÇÕES PRELIMINARES 
Art. 1º Esta Lei estabelece normas gerais de caráter nacional para 
o desenvolvimento, implementação e uso responsável de sistemas de 
inteligência artificial (IA) no Brasil, com o objetivo de proteger os direitos 
fundamentais e garantir a implementação de sistemas seguros e confiáveis, em 
benefício da pessoa humana, do regime democrático e do desenvolvimento 
científico e tecnológico. 
Art. 2º O desenvolvimento, a implementação e o uso de sistemas 
de inteligência artificial no Brasil têm como fundamentos: 
I – a centralidade da pessoa humana; 
II – o respeito aos direitos humanos e aos valores democráticos; 
III – o livre desenvolvimento da personalidade; 
IV – a proteção ao meio ambiente e o desenvolvimento 
sustentável; 
V – a igualdade, a não discriminação, a pluralidade e o respeito 
aos direitos trabalhistas; 
VI 

# **3. Dividindo Documentos - Splitting/Chunking**

Divisão do documento usando RecursiveCharacterTextSplitter.

In [5]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters


# Definir o separador de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=300, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)


# Exibir a quantidade de splits
print("Número de splits:", len(all_splits))


Número de splits: 94


In [6]:
all_splits

[Document(metadata={'source': 'https://legis.senado.leg.br/sdleg-getter/documento?dm=9347622&ts=1735605216274&disposition=inline', 'page': 0, 'start_index': 0}, page_content='SENADO FEDERAL  \nPROJETO DE LEI \nN° 2338, DE 2023\nDispõe sobre o uso da Inteligência Artificial.\nAUTORIA: Senador Rodrigo Pacheco (PSD/MG)\nPágina da matéria\nPágina 1 de 33 Avulso do PL 2338/2023'),
 Document(metadata={'source': 'https://legis.senado.leg.br/sdleg-getter/documento?dm=9347622&ts=1735605216274&disposition=inline', 'page': 1, 'start_index': 0}, page_content='PROJETO DE LEI Nº       , DE 2023 \nDispõe sobre o uso da Inteligência Artificial. \nO CONGRESSO NACIONAL decreta: \nCAPÍTULO I \nDISPOSIÇÕES PRELIMINARES \nArt. 1º Esta Lei estabelece normas gerais de caráter nacional para \no desenvolvimento, implementação e uso responsável de sistemas de \ninteligência artificial (IA) no Brasil, com o objetivo de proteger os direitos \nfundamentais e garantir a implementação de sistemas seguros e confiávei

In [7]:
print(all_splits[3].page_content)

VIII – a privacidade, a proteção de dados e a autodeterminação 
informativa; 
IX – a promoção da pesquisa e do desenvolvimento com a 
finalidade de estimular a inovação nos setores produtivos e no poder público; e 
X – o acesso à informação e à educação, e a conscientização sobre 
os sistemas de inteligência artificial e suas aplicações. 
Art. 3º O desenvolvimento, a implementação e o uso de sistemas 
de inteligência artificial observarão a boa-fé e os seguintes princípios: 
I – crescimento inclusivo, desenvolvimento sustentável e bem-
estar; 
II – autodeterminação e liberdade de decisão e de escolha; 
III – participação humana no ciclo da inteligência artificial e 
supervisão humana efetiva; 
IV – não discriminação; 
V – justiça, equidade e inclusão; 
VI – transparência, explicabilidade, inteligibilidade e 
auditabilidade; 
VII – confiabilidade e robustez dos sistemas de inteligência 
artificial e segurança da informação;


#**4. Indexando - Store**

Criação da vector store usando FAISS.

In [8]:
# Inicialização do modelo de embeddings
embeddings_model = OpenAIEmbeddings()

# Criação do vector store
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

# Criação do recuperador de documentos
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Realizando a busca
retrieved_docs = retriever.invoke("O que é sistema de inteligência artificial?")
len(retrieved_docs)

3

In [12]:
print(retrieved_docs[0].page_content)

XII – não maleficência e proporcionalidade entre os métodos 
empregados e as finalidades determinadas e legítimas dos sistemas de 
inteligência artificial. 
Art. 4º Para as finalidades desta Lei, adotam-se as seguintes 
definições: 
I – sistema de inteligência artificial: sistema computacional, com 
graus diferentes de autonomia, desenhado para inferir como atingir um dado 
conjunto de objetivos, utilizando abordagens baseadas em aprendizagem de 
máquina e/ou lógica e representação do conhecimento, por meio de dados de 
entrada provenientes de máquinas ou humanos, com o objetivo de produzir 
previsões, recomendações ou decisões que possam influenciar o ambiente 
virtual ou real; 
II – fornecedor de sistema de inteligência artificial: pessoa natural 
ou jurídica, de natureza pública ou privada, que desenvolva um sistema de 
inteligência artificial, diretamente ou por encomenda, com vistas a sua 
colocação no mercado ou a sua aplicação em serviço por ela fornecido, sob seu


#**5. Buscando e Recuperando Informações - Retrieve**

Criação do prompt template usando ChatPromptTemplate.

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

#**6. Gerando Respostas - Generate**

Criação da chain usando LangChain e realização de perguntas ao sistema.

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [16]:
for chunk in rag_chain.stream("O PROJETO DE LEI N° 2338 dispõe sobre o que?"):
    print(chunk, end="", flush=True)

O PROJETO DE LEI N° 2338 de 2023 dispõe sobre o uso da Inteligência Artificial, incluindo aspectos como direitos humanos, proteção ao meio ambiente e fiscalização. Ele estabelece normas e sanções administrativas relacionadas ao cumprimento dessas diretrizes.

In [17]:
for chunk in rag_chain.stream("O que são sistemas de inteligência artificial de alto risco ?"):
    print(chunk, end="", flush=True)

Sistemas de inteligência artificial de alto risco são aqueles utilizados em áreas críticas, como segurança de infraestruturas, educação e recrutamento, que exigem regulamentação e avaliação de impacto devido ao seu potencial de afetar significativamente a vida das pessoas. Eles estão sujeitos a normas de controle mais rigorosas para garantir a transparência e a mitigação de vieses.

In [18]:
for chunk in rag_chain.stream("Quais os direitos das pessoas afetadas por sistemas de inteligência artificial ?"):
    print(chunk, end="", flush=True)

As pessoas afetadas por sistemas de inteligência artificial têm direitos como o direito à informação prévia sobre interações, o direito à explicação sobre decisões tomadas, o direito de contestar essas decisões, e o direito à privacidade e proteção de dados. Além disso, devem ter acesso a participação humana nas decisões e proteção contra discriminação.

In [19]:
for chunk in rag_chain.stream("Qual a responsabilidade civil do fornecedor ou operador de sistema de inteligência artificial que cause dano patrimonial, moral, individual ou coletivo ?"):
    print(chunk, end="", flush=True)


O fornecedor ou operador de sistema de inteligência artificial que causar dano patrimonial, moral, individual ou coletivo é obrigado a repará-lo integralmente. Para sistemas de alto risco, a responsabilidade é objetiva, enquanto para os demais, a culpa é presumida, aplicando-se a inversão do ônus da prova em favor da vítima.

In [20]:
for chunk in rag_chain.stream("De quem é a obrigação para avaliar o impacto algorítmico de sistemas de inteligência artificial? E quais as etapas de avaliação de impacto?"):
    print(chunk, end="", flush=True)

A obrigação de avaliar o impacto algorítmico é do fornecedor ou operador do sistema de inteligência artificial, realizada por profissionais com conhecimentos técnicos, científicos e jurídicos. As etapas da avaliação incluem preparação, cognição do risco, mitigação dos riscos encontrados e monitoramento.